# `Nuqleon.Collections.Specialized`

Provides specialized collection types.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net50/Nuqleon.Collections.Specialized.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Collections.Specialized,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

## Bit arrays

Bit arrays provide an efficient storage format for arrays of bits of a specified length. Supported operations are getting and setting bits at the specified index. An example is shown below.

First, let's create a bit array of the specified size. This will store 24 bits, addressed with indexes 0 through 23.

In [ ]:
using System.Collections.Specialized;

IBitArray array = BitArrayFactory.Create(size: 24);

To simplify debugging, let's define a helper `Print()` function that simply enumerates over the bit array using the indexer, which returns a `bool` to indicate whether the bit is set or not. When a fresh bit array is allocated, all bits are set to `false`.

In [ ]:
void Print()
{
    var sb = new StringBuilder();

    for (int i = 0; i < array.Count; i++)
    {
        sb.Append(array[i] ? '1' : '0');
    }

    Console.WriteLine(sb.ToString());
}

Print();

000000000000000000000000


Next, let's explore the `SetAll` method which provides an easy way to toggle all bits at once.

In [ ]:
array.SetAll(true);

Print();

array.SetAll(false);

Print();

111111111111111111111111


000000000000000000000000


Finally, let's have a look at the indexer setter to toggle an individual bit.

In [ ]:
for (int i = 0; i < array.Count; i++)
{
    if (i % 2 == 0)
    {
        array[i] = true;
    }
}

Print();

101010101010101010101010


**Note:** More bit-wise operations could be added in the future. Because bit arrays are represented using an `IBitArray` interface, the use of operator overloading is problematic. Implementations of additional bit-wise operations can be done efficiently based on the underlying byte arrays that hold the bits.

## Enum dictionaries

Enum dictionaries provide a memory-efficient storage format for `Dictionary<K, V>` where `K` is an enum type. This can be useful to keep settings where the keys for settings are represented using an enum. Rather than storing the settings using `string`-based names, the use of an enum can be used to optimize the underlying storage format.

We'll first define an enum to be used as the key.

In [ ]:
enum Setting
{
    Bar,
    Foo,
    Qux,
    Baz,
}

Next, we'll use the `EnumDictionary.Create<K, V>` method to create an `IDictionary<K, V>` which is optimized for enum keys.

In [ ]:
IDictionary<Setting, string> settings = EnumDictionary.Create<Setting, string>();

The dictionary can be used like any other ordinary dictionary.

In [ ]:
settings.Add(Setting.Bar, "bar");
settings[Setting.Foo] = "foo";

foreach (var (key, value) in settings)
{
    Console.WriteLine($"{key} = {value}");
}

Bar = bar


Foo = foo


To illustrate the potential memory saving, we run a tiny benchmark in the next cell.

In [ ]:
using System.Threading;

static void Populate(IDictionary<Setting, string> settings)
{
    settings.Add(Setting.Bar, "bar");
    settings.Add(Setting.Foo, "foo");
    settings.Add(Setting.Qux, "qux");
    settings.Add(Setting.Baz, "baz");
}

static void Measure(string title, Func<IDictionary<Setting, string>> createDictionary, bool print)
{
    var t = new Thread(() =>
    {
        long mem = GC.GetAllocatedBytesForCurrentThread();

        IDictionary<Setting, string> settings = createDictionary();

        Populate(settings);

        if (print)
        {
            Console.WriteLine($"{title} - {GC.GetAllocatedBytesForCurrentThread() - mem} bytes allocated");
        }
    });
    t.Start();
    t.Join();
}

// NB: We run a warm-up iteration first that doesn't print to exclude allocation side-effects from e.g. running static initializers.

foreach (var print in new[] { false, true })
{
    Measure("EnumDictionary", () => EnumDictionary.Create<Setting, string>(), print);
    Measure("BCL Dictionary", () => new Dictionary<Setting, string>(), print);
}

EnumDictionary - 288 bytes allocated


BCL Dictionary - 464 bytes allocated
